In [9]:
# dataiter = iter(train_dataloader)
# images, masks = dataiter.next()

# # 데이터로더 증강데이터 시각화
# fig, axes = plt.subplots(2, 8, figsize=(12, 6))

# for i in range(8):
#     axes[0, i].imshow(images[i].permute(1, 2, 0))
#     axes[0, i].set_title(f"TrainSample\n{images[i].shape[1]} x {images[i].shape[2]}")

#     axes[1, i].imshow(masks[i].squeeze(), cmap='gray')
#     axes[1, i].set_title(f"Mask\n{masks[i].shape[0]} x {masks[i].shape[1]}")

#     #격자 제거
#     axes[0, i].axis('off')
#     axes[1, i].axis('off')

# plt.tight_layout()
# plt.show()

In [10]:
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
from IPython.display import clear_output 
import json


In [11]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [12]:
import bisect

def is_number_in_list(lst, target):
    index = bisect.bisect_left(lst, target)
    if index != len(lst) and lst[index] == target:
        return True
    return False

In [15]:
# 은동이 분할코드

transform = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)

def get_list_idx(idx, data):
    img_path = "../data" + data.iloc[idx, 1][1:]
    image = cv2.imread(img_path)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    mask_rle = data.iloc[idx, 2]
    mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

    # 이미지 분할
    patches = split_image(image, 224, 224)
    mask_patches = split_image(mask, 224, 224)

    return patches, mask_patches

def split_image(image, patch_size, stride):
        patches = []
        h, w = image.shape[:2]
        for i in range(0, h-patch_size+1, stride):
            for j in range(0, w-patch_size+1, stride):
                if i+patch_size > h or j+patch_size > w:
                     continue
                patch = image[i:i+patch_size, j:j+patch_size]
                patches.append(patch)
        return patches


with open("../baseline/train_trash.json","r") as js:
    json_file = json.load(js)
        
train_trash = json_file["must"]
train_quarter = json_file["quarter"]

with open("../baseline/val_trash.json","r") as js:
    json_file = json.load(js)

val_trash = json_file["must"]
val_quarter = json_file["quarter"]

trash_sum_list = train_trash + train_quarter + val_trash + val_quarter

trash_sum_list = sorted(trash_sum_list)

data = pd.read_csv('../data/train.csv')
idx = 0
for i in tqdm(range(len(data))):
    #clear_output()             
    if is_number_in_list(trash_sum_list, i):
        #assert False
        continue

    temp = get_list_idx(i, data)
    for j,v in enumerate(temp[0]):
        cv2.imwrite(f"../split_data_224_trash/train_img/{idx}_{j}.png",v)
    for j,v in enumerate(temp[1]):
        cv2.imwrite(f"../split_data_224_trash/train_mask/{idx}_{j}.png",v)
    idx += 1
    
print(len(data)-len(trash_sum_list), idx-1)

100%|██████████| 7140/7140 [19:06<00:00,  6.23it/s]  

6717 6719


0 ~ 7139 _ 0~15 . png